In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0
%reload_ext autoreload
!pip install -U imbalanced-learn
import threadpoolctl
threadpoolctl.threadpool_info()
%pip install optuna

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessing
from preprocessing import *
import warnings
warnings.filterwarnings("ignore")

In [0]:
data= pd.read_csv("emi_prediction_dataset.csv", low_memory=False)

In [0]:
pre=quality_check(data, Prediction=False)

In [0]:
n_data = pre.final_processsing().emi_pred

In [0]:
target_cols=['emi_eligibility', 'max_monthly_emi']

In [0]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [0]:
def data_prepare(data, datatype="type1", target_type=None):
    final_columns_v1 = [
    'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents', 'school_fees',
       'college_fees', 'travel_expenses', 'groceries_utilities',
       'other_monthly_expenses', 'existing_loans', 'current_emi_amount',
       'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario',
       'requested_amount', 'requested_tenure']
    final_columns_v2=[
     'gender', 'marital_status', 'education', 'monthly_salary',
       'employment_type', 'years_of_employment', 'company_type', 'house_type',
       'monthly_rent', 'family_size', 'dependents',  'existing_loans', 'credit_score', 'emi_scenario',
       'requested_amount', 'requested_tenure', 'Total_Fixed_expenses',
       'Total_Other_expenses','Savings']
    if datatype=="type1":
        if target_type==None:
            final_data=data[final_columns_v1]
        elif target_type=="regression":
            final_data=data[final_columns_v1]
            final_data['max_monthly_emi']=data["max_monthly_emi"]
        elif target_type=="classification":
            final_data=data[final_columns_v1]
            final_data['emi_eligibility']=data["emi_eligibility"]
        else:
            print("Please enter correct target type")
    elif datatype=="type2":
        if target_type==None:
            final_data=data[final_columns_v2]
        elif target_type=="regression":
            final_data=data[final_columns_v2]
            final_data['max_monthly_emi']=data["max_monthly_emi"]
        elif target_type=="classification":
            final_data=data[final_columns_v2]
            final_data['emi_eligibility']=data["emi_eligibility"]
    return final_data

In [0]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
def data_split(x_data,y_data, rebalance = False):
    X=x_data
    y=y_data
    if rebalance==True:
        sm = SMOTE(random_state=42)
        X, y = sm.fit_resample(X, y)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    return x_train, x_test, y_train, y_test

In [0]:
n_data.columns

In [0]:
n_data["Savings"] = n_data["bank_balance"] + n_data["emergency_fund"]

In [0]:
n_data.columns

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")
import optuna

In [0]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, matthews_corrcoef, precision_recall_fscore_support
def logging(y_test, pred, model_name="model"):
    # --- 1️⃣ Core Metrics ---
    mlflow.log_metric("accuracy", accuracy_score(y_test, pred))
    mlflow.log_metric("f1_score_weighted", f1_score(y_test, pred, average='weighted'))
    mlflow.log_metric("recall_weighted", recall_score(y_test, pred, average='weighted'))
    mlflow.log_metric("precision_weighted", precision_score(y_test, pred, average='weighted'))

    balanced_acc = balanced_accuracy_score(y_test, pred)
    kappa = cohen_kappa_score(y_test, pred)
    mcc = matthews_corrcoef(y_test, pred)

    mlflow.log_metric("balanced_accuracy", balanced_acc)
    mlflow.log_metric("cohen_kappa", kappa)
    mlflow.log_metric("mcc", mcc)

    # --- 2️⃣ Confusion Matrix ---
    cm = confusion_matrix(y_test, pred, labels=np.unique(y_test))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_test))

    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(cmap='Blues', ax=ax, colorbar=False)
    plt.title(f"Confusion Matrix - {model_name}")

    mlflow.log_figure(fig, f"{model_name}_confusion_matrix.png")
    plt.close(fig)

    # --- 3️⃣ Per-Class Metrics ---
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, pred, average=None, labels=np.unique(y_test)
    )

    for i, label in enumerate(np.unique(y_test)):
        mlflow.log_metric(f"precision_class_{label}", prec[i])
        mlflow.log_metric(f"recall_class_{label}", rec[i])
        mlflow.log_metric(f"f1_class_{label}", f1[i])

    # --- 4️⃣ Log CM as artifact ---
    #mlflow.log_artifact(figure=f"{model_name}_confusion_matrix.png")

    print(f"✅ Logged metrics and confusion matrix for {model_name}")

In [0]:
import pickle

In [0]:
with open("assets/v1_scaler.pkl", "rb") as f:
    v1_scaler = pickle.load(f)
with open("assets/v2_scaler.pkl", "rb") as f:
    v2_scaler = pickle.load(f)

In [0]:
data_type1=data_prepare(n_data, target_type="classification", datatype="type1")
data_type2=data_prepare(n_data, target_type="classification", datatype="type2")


In [0]:
X_dtype1=data_type1.drop("emi_eligibility", axis=1)
y_dtype1=data_type1["emi_eligibility"]
X_dtype2=data_type2.drop("emi_eligibility", axis=1)
y_dtype2=data_type2["emi_eligibility"]
X_dtype1=pd.DataFrame(v1_scaler.transform(X_dtype1),columns=X_dtype1.columns)
X_dtype2=pd.DataFrame(v2_scaler.transform(X_dtype2), columns=X_dtype2.columns)



In [0]:

def objective_rf(trial):
    run_name="Randomforest_optimization_run"
    model_name="Random_Forest_Classifier"
    with mlflow.start_run(nested=True, run_name=run_name):
        x_train, x_test, y_train, y_test = data_split(X_dtype1, y_dtype1, rebalance = True)  # Nested run under Optuna main run
        # Suggest hyperparameters
        n_estimators = trial.suggest_int("n_estimators", 300, 600, step=100)
        max_depth = trial.suggest_int("max_depth", 5, 20)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
        bootstrap = trial.suggest_categorical("bootstrap", [True, False])

        # Initialize model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            class_weight="balanced",
            n_jobs=-1,
            random_state=42
        )

        model.fit(x_train, y_train)
        preds = model.predict(x_test)

        #acc = accuracy_score(y_test, preds)
        f1 = f1_score(y_test, preds, average='weighted')

        # Log params & metrics to MLflow
        mlflow.log_params({
            "n_estimators": n_estimators,
            "max_depth": max_depth,
            "min_samples_split": min_samples_split,
            "min_samples_leaf": min_samples_leaf,
            "bootstrap": bootstrap
        })
        logging(y_test, preds, model_name=model_name)


        return f1

In [0]:
import mlflow
import optuna

In [0]:
try:
    mlflow.create_experiment("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/Random_Forest_Classifier_Optuna")
    print("Experiment created")
except:
    print("Experiment already exists")
mlflow.set_experiment("/Workspace/Users/sushantkashikar1@gmail.com/mlflow/Random_Forest_Classifier_Optuna")

with mlflow.start_run(run_name="Random_Forest_Classifier"):
    study_rf = optuna.create_study(direction="maximize", study_name="Random_forest_Study_v3")
    try:
        study_rf.optimize(objective_rf, n_trials=20)
    finally:
        print("✅ Best RandomForest F1:", study_rf.best_value)
        mlflow.log_params(study_rf.best_params)

In [0]:
!pip install lightgbm